# Libs

In [ ]:
suppressPackageStartupMessages({
    library(tessera)

    ## Consider adding dependencies
    # library(data.table)

    ## Remove these dependencies 
    # library(RSpectra)
    # library(sf) ## my own polygons, sf only for plotting? 
    # library(mclust) ## mclustBIC. Just use RcppArmadillo's version 

    ## Move to suggests 
    ## (for vignette only, not method)
    library(ggplot2)
    library(ggthemes)
    library(viridis)
    library(glue)
    library(patchwork)
    library(ggplot2)
    library(ComplexHeatmap)
})

fig.size <- function(h, w) {
    options(repr.plot.height = h, repr.plot.width = w)
}



# Data

Small sample MERFISH dataset from Chen et al: 

https://www.biorxiv.org/content/10.1101/2023.04.04.535379v1.abstract


In [ ]:
data('tessera_warmup')
counts = tessera_warmup$counts
meta_data = tessera_warmup$meta_data

In [ ]:
meta_vars_include = c('type')


In [ ]:
fig.size(8, 8)
ggplot() + 
    geom_point(data = meta_data, aes(X, Y, color = type)) + 
    theme_void() + 
    scale_color_tableau() + 
    coord_sf(expand = FALSE) + 
    NULL


Some coarse grained cell types are predefined here, to help interpret the tiles we get below. 

In [ ]:
table(meta_data$type)

# Get Tiles

Run the Tessera algorithm to get tiles in one function. The result is returns in two structures: 

1) dmt: cell-level information. 
2) aggs: tile-level information. 

The two are tied together through `dmt$pts$agg_id`


In [ ]:
res = GetTiles(
    X = meta_data$X, 
    Y = meta_data$Y, 
    counts = counts, 
    meta_data = meta_data, 
    meta_vars_include = meta_vars_include,
    max_npts = 50, 
    min_npts = 5,
    smooth_distance = 'projected', smooth_similarity = 'projected' ## Typically more discriminative 
    # smooth_distance = 'none', smooth_similarity = 'none' ## DEFAULT
)
dmt = res$dmt
aggs = res$aggs


In [ ]:
fig.size(10, 30)
purrr::map(1:3, function(i) {
    ggplot(cbind(aggs$meta_data, val=aggs$pcs[, i])) + 
        geom_sf(aes(geometry = shape, fill = val)) + 
        theme_void(base_size = 16) + 
        coord_sf(expand = FALSE) + 
        scale_fill_gradient2_tableau() + 
        guides(color = 'none') + 
        labs(title = glue('PC{i}')) + 
        NULL 
}) %>% 
    purrr::reduce(`|`)


# Cluster and label tiles 

Let's treat each aggregate as a unit of analysis. 


In [ ]:
library(Seurat)

In [ ]:
obj = Seurat::CreateSeuratObject(
    counts = aggs$counts, 
    meta.data = tibble::column_to_rownames(data.frame(dplyr::select(aggs$meta_data, -shape)), 'id')
)

## Seurat doesn't do sf shapes well 
obj@meta.data$shape = aggs$meta_data$shape

Do all the typical steps for Seurat clustering. 

In [ ]:
.verbose = FALSE
obj = obj %>% 
    NormalizeData(normalization.method = 'LogNormalize', scale.factor = median(obj@meta.data$nCount_RNA), verbose = .verbose) %>% 
    ScaleData(features = rownames(obj), verbose = .verbose) %>% 
    RunPCA(npcs = 20, features = rownames(obj), verbose = .verbose) %>% 
    RunUMAP(verbose = .verbose, dims = 1:20) %>% 
    Seurat::FindNeighbors(features = 1:20, reduction = 'pca', verbose = .verbose) %>% 
    Seurat::FindClusters(verbose = .verbose, resolution = c(2))

Let's see the aggregate clusters in UMAP and physical space. 

In [ ]:
p1 = DimPlot(obj, reduction = 'umap', group.by = 'seurat_clusters') + scale_color_tableau('Classic 10') 
p2 = ggplot(obj@meta.data) + 
    geom_sf(aes(geometry = shape, fill = seurat_clusters)) + 
    theme_void(base_size = 16) + 
    coord_sf(expand = FALSE) + 
    scale_fill_tableau('Classic 10') + 
    # guides(color = 'none', fill = 'none') + 
    NULL 

fig.size(6, 12)
(p1 | p2) + plot_layout(widths = c(1, 1))

# Transfer agg information to cells

In [ ]:
dmt$pts$spatial_cluster = obj@meta.data$seurat_clusters[dmt$pts$agg_id]

In [ ]:
p1 = ggplot() + 
    geom_sf(data = obj@meta.data, aes(geometry = shape), fill = NA) + 
    geom_point(data = dmt$pts, aes(X, Y, color = type)) + 
    scale_color_tableau() + 
    theme_void() + 
    coord_sf(expand = FALSE) + 
    NULL
p2 = ggplot() + 
    geom_sf(data = obj@meta.data, aes(geometry = shape, fill = seurat_clusters), alpha = .2) + 
    geom_point(data = dmt$pts, aes(X, Y, color = spatial_cluster)) + 
    scale_color_tableau('Classic 10') + 
    scale_fill_tableau('Classic 10') + 
    theme_void() + 
    guides(fill = 'none') + 
    coord_sf(expand = FALSE) + 
    NULL
fig.size(8, 20)
p1 | p2


In [ ]:
fig.size(5, 8)
dmt$pts %>% 
    with(table(type, spatial_cluster)) %>% 
    prop.table(2) %>% 
    t %>% scale %>% t %>% 
    Heatmap()